# DASi Submission

In [1]:
from aqbt.tools import config_to_sessions
from aqbt.tools import parse_config
import toml

def config(config_path):
    with open(config_path, "r") as f:
        return parse_config(toml.load(f))


def sessions(config):
    return config_to_sessions(config)


sessions = sessions(config('config.toml'))
sessions

{'default': {'aquarium': <AqSession(name=None, AqHTTP=<AqHTTP(user='vrana', url='http://52.27.43.242')>), parent=4516379384)>,
  'benchling': <benchlingapi.session.Session at 0x133ee84f0>,
  'registry': <aqbt.aquarium.registry.LabDNARegistry at 0x133f13790>}}

#### Post on Benchling

In [2]:
benchling = sessions['default']['benchling']

folder = benchling.Folder.find_by_name("2020Campaign")

In [3]:
from benchlingapi import __version__
__version__

'2.1.12'

In [4]:
from aqbt.bioadapter import convert
from aqbt import biopython
from glob import glob

records = biopython.load_genbank_glob(glob('output_plasmids/*.gb'))

seqs = []
for record in records:
    data = convert(record, to='benchling_dna_json')
    data['folder_id'] = folder.id
    seq = benchling.DNASequence.load(data)
    seqs.append(seq)

In [5]:
# import tqdm

# for seq in seqs:
#     print(seq.name)
#     seq.save()

In [7]:
sessions['default']['registry'].session.using_cache = True

In [8]:
import pickle
from os.path import abspath
from os.path import dirname
from os.path import isfile
from os.path import join

from dasi import LibraryDesign

from aqbt.bioadapter import convert
from aqbt.design.dasi.dfs import KlavinsLabDnaDb


here = abspath(dirname("."))
DEFAULT_FILEPATH = join(here, "inventory.df")

from aqbt.design.dasi.dasi_to_aquarium import design_from_benchling_links, inventory_df

def inventory_df(registry=None, force_new: bool = False, filepath: str = None):
    if filepath is None:
        filepath = DEFAULT_FILEPATH
    df = None
    if isfile(filepath) and not force_new:
        with open(filepath, "rb") as f:
            try:
                return pickle.load(f)
            except Exception as e:
                print("Could not load pickled file.")
                print(e)
    if df is None:
        db = KlavinsLabDnaDb(registry)
        db.build(dna_limit=None, primer_limit=None)
        print(db.df['is_available'])
        df = db.df[db.df["is_available"] == True]
        df.drop("benchling_sequence", axis=1, inplace=True)
        with open(filepath, "wb") as f:
            pickle.dump(df, f)

    # clean
    df["trashed"] = [s.name.lower().startswith("trashed") for s in df["sample"]]
    df = df[df["is_available"] == True]
    df = df[~df["trashed"]]
    return df

df = inventory_df(sessions['default']['registry'])

In [9]:
df.columns

Index(['entity_registry_id', 'is_available', 'is_circular', 'record',
       'record_uuid', 'sample', 'sample_id', 'sample_type', 'sequence_hash'],
      dtype='object')

In [ ]:
df = df[df['is_available'] == True]
# df = df[~df['trashed']]

def _get_records(st):
    return list(df[df["sample_type"] == st]["record"])

fragments = _get_records('Fragment')
plasmids = _get_records("Plasmid")
primers = _get_records("Primer")


biopython.make_cyclic(records)
biopython.make_linear(fragments)
biopython.make_linear(primers)
biopython.make_linear(plasmids)

span_cost = dasi.cost.cached_span_cost()
design = LibraryDesign(span_cost=span_cost)
design.FAVOR_SHARED_SEQUENCES = 10.0
design.add_materials(
    primers=primers, fragments=fragments, templates=plasmids, queries=records
)
design.logger.set_level("DEBUG")
design.run(n_paths=1)


                                             
  0%|          | 0/5 [00:00<?, ?it/s]

INFO - ProgressLoggable(LibraryDesign(id=5674661680)) - 2020-07-20 15:40:49,601 - Running blast


                                             
  0%|          | 0/5 [00:17<?, ?it/s]

INFO - LibraryDesign(id=5674661680) - 2020-07-20 15:41:06,874 - Number of template matches: 45711


                                             
  0%|          | 0/5 [00:18<?, ?it/s]

...

INFO - ProgressLoggable(LibraryDesign(id=5674661680)) - 2020-07-20 15:41:18,710 - Running shared fragment blast
INFO - LibraryDesign(id=5674661680) - 2020-07-20 15:41:18,718 - === Expanding shared library fragments ===


                                             
 20%|██        | 1/5 [00:32<01:56, 29.12s/it]

INFO - LibraryDesign(id=5674661680) - 2020-07-20 15:41:21,974 - Found 5026 shared alignments between the queries


                                             
 80%|████████  | 4/5 [00:33<00:20, 20.79s/it]

INFO - ProgressLoggable(LibraryDesign(id=5674661680)) - 2020-07-20 15:41:22,826 - Finding shared clusters


In [13]:
import dasi

In [18]:
span_cost = dasi.cost.cached_span_cost()